In [1]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

d:\Crowd-Funding\Crowd-Funding\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [3]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment. Please set it in your .env file.")

# Configuration
MODEL = 'gpt-4o-mini'
db_name = 'vector_db'

In [4]:
folders = glob.glob('knowledge_base/*')
text_loader_kwargs = {'encoding': 'utf8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob='**/*.md', loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata['doc_type'] = doc_type
        documents.append(doc)

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [6]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [8]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)

In [9]:
qa_template = """You are an AI assistant for a crowdfunding platform founded by Saad Zaidi, Waliuddin Ahmed, and Sajjad Ahmed - Computer Science students at FAST University.

YOUR ROLE:
You help users understand and navigate our crowdfunding platform where people can donate to verified causes through multiple payment methods including credit/debit cards, PayPal, EasyPaisa, and JazzCash.

INSTRUCTIONS:
- For greetings: Respond warmly and briefly
- For questions: Use the context below to answer accurately
- If uncertain: Say "I don't have that specific information in my knowledge base"
- Be professional, concise, and helpful
- Break down complex processes into clear steps

CONTEXT:
{context}

QUESTION: {question}

ANSWER:"""

QA_PROMPT = PromptTemplate(
    template=qa_template,
    input_variables=["context", "question"]
)

In [10]:
llm = ChatOpenAI(
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0.7
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True,
    output_key='answer'
)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": QA_PROMPT}
)

C:\Users\SAAD\AppData\Local\Temp\ipykernel_55748\3677067304.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [11]:
small_talk = {
    "hi": "Hello! How can I help you today?",
    "hello": "Hi there! Welcome to our crowdfunding platform.",
    "hey": "Hey! What can I assist you with?",
    "thanks": "You're welcome!",
    "thank you": "Happy to help!",
    "bye": "Goodbye! Have a great day!",
    "goodbye": "Take care! Feel free to come back anytime."
}

In [12]:
def chat(message, history):
    if message.lower().strip() in small_talk:
        return small_talk[message.lower().strip()]
    
    try:
        result = conversation_chain.invoke({"question": message})
        return result['answer']
    except Exception as e:
        print(f"Error in chat: {e}")
        return "I apologize, but I encountered an error processing your request. Please try again or rephrase your question."


In [13]:
demo = gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Crowdfunding Platform Assistant",
    description="Ask me anything about campaigns, donations, payment methods, or platform policies!",
    examples=[
        "How do I create a campaign?",
        "What payment methods do you support?",
        "Tell me about the founders",
        "How does the campaign approval process work?",
        "What are the platform fees?"
    ],
    theme=gr.themes.Soft()
)

In [14]:
demo.launch(inbrowser=True, share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://233713e1eef7c4a8f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
* Running on public URL: https://233713e1eef7c4a8f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
